In [1]:
import torch

In [2]:
torch.__version__

'2.0.0+cu118'

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.zeros(1).cuda(1)

tensor([0.], device='cuda:1')

In [5]:
a = 0.123456789
print('{:.5f}'.format(a))

0.12346


In [6]:
import torch
from thop import profile
from torchvision.models import resnet50
from network.unet_tf import UNetTF
from network.unet import UNet


model = resnet50()
input_data = torch.randn(1, 3, 224, 224)  # 输入数据的形状

# 使用thop.profile计算FLOPs
flops, params = profile(model, inputs=(input_data,), verbose=False)
print(f"ResNet50 FLOPs: {flops / 1e9} G")
print(f"ResNet50 Parameters: {params / 1e6} M")

save_mode_path = '../model/V1_contrast_fold1/scribble/best_net_dice.pth'
model_pclmix = UNetTF(in_channels=1, classes=4).cuda()
model_pclmix.load_state_dict(torch.load(save_mode_path))

model_trimix = UNet(in_chns=1, class_num=4).cuda()
input_data = torch.randn(1, 1, 224, 224).cuda()  # 输入数据的形状

# 使用thop.profile计算FLOPs
flops_pclmix, params_pclmix = profile(model_pclmix, inputs=(input_data,), verbose=False)
print(f"PCLMix FLOPs (without transformer decoder): {flops_pclmix / 1e9} G")
print(f"PCLMix Parameters (without transformer decoder): {params_pclmix / 1e6} M")

flops_trimix, params_trimix = profile(model_trimix, inputs=(input_data,), verbose=False)
print(f"TriMix FLOPs: {flops_trimix*3 / 1e9} G")
print(f"TriMix Parameters: {params_trimix*3 / 1e6} M")


ResNet50 FLOPs: 4.133742592 G
ResNet50 Parameters: 25.557032 M


/home/duweihong/anaconda3/envs/swinmix/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


PCLMix FLOPs (without transformer decoder): 8.140453888 G
PCLMix Parameters (without transformer decoder): 32.515268 M
TriMix FLOPs: 6.898096128 G
TriMix Parameters: 5.441292 M


ResNet50 FLOPs: 4.133742592 G
ResNet50 Parameters: 25.557032 M
PCLMix FLOPs (without transformer decoder): 8.140453888 G
PCLMix Parameters (without transformer decoder): 32.515268 M
TriMix FLOPs: 6.898096128 G
TriMix Parameters: 5.441292 M